# Challenge Part 2, Narrow Travel Searches Based on Temp and Rain

In [1]:
import pandas as pd
import requests
import gmaps

from config import g_key

In [8]:
city_data_df = pd.read_csv("./Data/WeatherPy_Database.csv")
city_data_df

,City_ID,City,Cloudiness,Country,Current Description,Humidity,Lat,Lng,Max Temp,Rain (inches),Snow (inches),Wind Speed
0,0,ilebo,50,CD,scattered clouds,58,-4.32,20.58,67.89,0.0,0,0.92
1,1,makakilo city,20,US,few clouds,57,21.35,-158.09,82.40,0.0,0,13.87
2,2,calvinia,0,ZA,clear sky,25,-31.47,19.78,64.40,0.0,0,17.22
3,3,hobart,90,AU,mist,93,-42.88,147.33,55.00,0.0,0,8.05
4,4,dikson,13,RU,few clouds,77,73.51,80.55,39.24,0.0,0,9.75
...,...,...,...,...,...,...,...,...,...,...,...,...
569,569,vysokogornyy,100,RU,overcast clouds,70,50.12,139.15,55.80,0.0,0,6.69
570,570,estelle,90,US,overcast clouds,100,29.85,-90.11,77.00,0.0,0,9.86
571,571,puksoozero,0,RU,clear sky,65,62.59,40.61,42.67,0.0,0,5.68
572,572,batagay-alyta,70,RU,broken clouds,24,67.80,130.41,83.61,0.0,0,9.35


In [9]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [10]:
# Max Temp Criteria
min_temp = float(input("What is the deired minimum temperature for your vacation?"))
max_temp = float(input("What is the deired maximum temperature for your vacation?"))

What is the deired minimum temperature for your vacation?60
What is the deired maximum temperature for your vacation?105


In [11]:
# Ask customer about rain preference
rain_amount = input("Do you want it to be raining? (yes/no)")

Do you want it to be raining? (yes/no)no


In [12]:
# Ask customer about snow preference
snow_amount = input("Do you want it to be snowing? (yes/no)")

Do you want it to be snowing? (yes/no)no


In [13]:
if rain_amount == "no" and snow_amount == "no":
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                          (city_data_df["Max Temp"] >= min_temp) &
                                          (city_data_df["Rain (inches)"] == 0) &
                                          (city_data_df["Snow (inches)"] == 0)]
elif rain_amount == "no" and snow_amount == "yes":
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                          (city_data_df["Max Temp"] >= min_temp) &
                                          (city_data_df["Rain (inches)"] > 0) &
                                          (city_data_df["Snow (inches)"] > 0)]
elif rain_amount == "yes" and snow_amount == "no":
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                          (city_data_df["Max Temp"] >= min_temp) &
                                          (city_data_df["Rain (inches)"] > 0) &
                                          (city_data_df["Snow (inches)"] == 0)]
else:
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                          (city_data_df["Max Temp"] >= min_temp) &
                                          (city_data_df["Rain (inches)"] > 0) &
                                          (city_data_df["Snow (inches)"] > 0)]

In [14]:
filtered_cities_df.head(2)

,City_ID,City,Cloudiness,Country,Current Description,Humidity,Lat,Lng,Max Temp,Rain (inches),Snow (inches),Wind Speed
0,0,ilebo,50,CD,scattered clouds,58,-4.32,20.58,67.89,0.0,0,0.92
1,1,makakilo city,20,US,few clouds,57,21.35,-158.09,82.40,0.0,0,13.87


In [18]:
hotel_df = filtered_cities_df[["City","Country","Max Temp","Lat","Lng"]].copy()
hotel_df['Hotel Name'] = " "
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,ilebo,CD,67.89,-4.32,20.58,
1,makakilo city,US,82.40,21.35,-158.09,
2,calvinia,ZA,64.40,-31.47,19.78,
7,boulsa,BF,84.00,12.67,-0.58,
8,launceston,AU,60.01,-41.45,147.17,


In [22]:
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate Through
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotels = requests.get(base_url,params=params).json()
    
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
    except IndexError:
        print("Hotel Not Found... skipping...")

Hotel Not Found... skipping...
Hotel Not Found... skipping...
Hotel Not Found... skipping...
Hotel Not Found... skipping...
Hotel Not Found... skipping...
Hotel Not Found... skipping...
Hotel Not Found... skipping...
Hotel Not Found... skipping...
Hotel Not Found... skipping...
Hotel Not Found... skipping...
Hotel Not Found... skipping...
Hotel Not Found... skipping...
Hotel Not Found... skipping...
Hotel Not Found... skipping...
Hotel Not Found... skipping...
Hotel Not Found... skipping...
Hotel Not Found... skipping...
Hotel Not Found... skipping...
Hotel Not Found... skipping...
Hotel Not Found... skipping...
Hotel Not Found... skipping...
Hotel Not Found... skipping...
Hotel Not Found... skipping...
Hotel Not Found... skipping...
Hotel Not Found... skipping...
Hotel Not Found... skipping...
Hotel Not Found... skipping...
Hotel Not Found... skipping...
Hotel Not Found... skipping...
Hotel Not Found... skipping...
Hotel Not Found... skipping...
Hotel Not Found... skipping...
Hotel No

In [23]:
output_data_file = "./Data/WeatherPy_Vacation.csv"
hotel_df.to_csv(output_data_file,index_label="City_ID")

In [24]:
vacation_df = pd.read_csv("./Data/WeatherPy_Vacation.csv")
vacation_df.head()

,City_ID,City,Country,Max Temp,Lat,Lng,Hotel Name
0,0,ilebo,CD,67.89,-4.32,20.58,Hôtel des palmes
1,1,makakilo city,US,82.40,21.35,-158.09,Marriott's Ko Olina Beach Club
2,2,calvinia,ZA,64.40,-31.47,19.78,Die Lemoenhuis
3,7,boulsa,BF,84.00,12.67,-0.58,Boulsa tintin
4,8,launceston,AU,60.01,-41.45,147.17,Hotel Grand Chancellor Launceston


In [26]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Hotel Name</dt>{Hotel Name} at {Max Temp}</dd>
"""
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]
locations = vacation_df[["Lat","Lng"]]

In [29]:
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))